In [1]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

transform = transforms.Compose( # composing several transforms together
    [transforms.ToTensor(), # to tensor object
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # mean = 0.5, std = 0.5

# set batch_size
batch_size = 4

# set number of workers
num_workers = 2

# load train data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [2]:
class Net(nn.Module):
    ''' Models a simple Convolutional Neural Network'''
	
    def __init__(self):
        super(Net, self).__init__()
	# 3 input image channel, 6 output channels, 
	# 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
	# Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5) 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
sample = next(iter(trainloader))

AttributeError: 'list' object has no attribute 'shape'

In [8]:
inputs, labels = sample
labels

tensor([4, 8, 5, 1])

In [12]:

optimizer.zero_grad()

# forward + backward + optimize
outputs = net(inputs)
_, indices = torch.max(outputs, 1)
indices

tensor([6, 6, 6, 6])

In [13]:
outputs

tensor([[-0.0164, -0.0374, -0.0895, -0.0345,  0.0243,  0.0585,  0.0728, -0.0640,
         -0.0766, -0.0260],
        [-0.0058, -0.0459, -0.0959, -0.0396,  0.0344,  0.0671,  0.0759, -0.0691,
         -0.0715, -0.0195],
        [-0.0025, -0.0425, -0.1001, -0.0422,  0.0408,  0.0661,  0.0775, -0.0579,
         -0.0747, -0.0199],
        [ 0.0022, -0.0325, -0.0983, -0.0432,  0.0289,  0.0599,  0.0838, -0.0710,
         -0.0721, -0.0218]], grad_fn=<AddmmBackward0>)

In [14]:
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()
loss

tensor(2.2906, grad_fn=<NllLossBackward0>)

In [15]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
input



tensor([[-1.5247, -0.5842, -0.9288,  0.5530,  0.1743],
        [ 0.5449,  1.5820,  1.1140,  1.9843,  0.5505],
        [-0.6597,  1.2930,  1.3542,  0.7932,  2.0719]], requires_grad=True)

In [16]:
target

tensor([1, 2, 2])

In [17]:
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()
input

tensor([[ 0.4284,  0.0989, -0.3677, -1.3392,  1.4344],
        [-0.0558, -0.4223, -0.2999, -1.4364, -0.2017],
        [-1.7763, -0.9488, -1.5666, -0.5978, -0.1989]], requires_grad=True)

In [18]:
target

tensor([[0.0685, 0.4091, 0.1139, 0.2584, 0.1501],
        [0.1065, 0.2762, 0.0805, 0.3167, 0.2202],
        [0.0388, 0.2279, 0.1018, 0.0873, 0.5443]])

In [27]:
import torch
import pandas as pd
from model import Net, Trainer
from data_manager import LoaderFactory

MODEL_PATH = '../saved_model/model.pth'


TRAIN_DATA = '../data/train.csv'
TEST_DATA = '../data/test.csv'
TRAIN_IMG = '../data/train_test_data/train/'


    # load data
def split_data_frame(df, train_frac=0.8):
    train = df.sample(frac=train_frac).reset_index()
    test = df.drop(train.index).reset_index()
    return train, test
    
train_df, test_df = split_data_frame(pd.read_csv(TRAIN_DATA))
loader_fact = LoaderFactory()
train_DL = loader_fact.get_data_loader('../data/', train_df, train=True)
test_DL = loader_fact.get_data_loader('../data/', train_df, train=True)

In [28]:
train_DL.dataset.labels

0       0
1       0
2       2
3       2
4       2
       ..
1366    0
1367    1
1368    2
1369    0
1370    1
Name: label, Length: 1371, dtype: int64

In [29]:
next(iter(train_DL))

{'image': tensor([[[[-1.0000, -0.8588, -0.8431,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -0.8431, -0.8196,  ..., -0.8667, -0.8510, -0.8510],
           [-1.0000, -0.8510, -0.8431,  ..., -0.8667, -0.8667, -0.8588],
           ...,
           [-0.7961, -0.7882, -0.7804,  ..., -0.8588, -0.8667, -1.0000],
           [-0.7882, -0.7804, -0.7882,  ..., -0.8353, -0.8510, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -0.8196, -0.8353, -1.0000]],
 
          [[-1.0000, -0.7725, -0.7490,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -0.7490, -0.7333,  ..., -0.7961, -0.7882, -0.7804],
           [-1.0000, -0.7647, -0.7490,  ..., -0.7961, -0.7961, -0.7882],
           ...,
           [-0.7255, -0.7020, -0.6863,  ..., -0.7647, -0.7725, -1.0000],
           [-0.7176, -0.6941, -0.6941,  ..., -0.7333, -0.7569, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -0.7176, -0.7333, -1.0000]],
 
          [[-1.0000, -0.8510, -0.8275,  ..., -1.0000, -1.0000, -1.0000],
    

In [ ]:
   # train splited data
    splited_net = Net((3, 332, 332), classes=3)
    trainer = Trainer(splited_net, train_DL)
    trainer.train()
    y_pred, y_true = splited_net.predict(test_DL)
    score = splited_net.score(y_pred, y_true)
    print(f'Finished training, F1 score (macro) of {score}')
        # train
        # predict    
        # calcualte f1 score

    # train whole data
    # save model
        # torch.save(net.state_dict(), MODEL_PATH)    
    # predict+save predictions